In [1]:
import configparser
import numpy as np
import os
import copy

In [2]:
from trainlib.config import Config
from trainlib.SimpleModel import SimpleModel
from trainlib.ModelFactory import ModelFactory
from trainlib.ConfigFileHandler import ConfigFileHandler
from trainlib.ModelCollectionConfigFileHandler import ModelCollectionConfigFileHandler

Using TensorFlow backend.


Welcome to JupyROOT 6.10/09


In [3]:
def dict_to_dirname(in_dict):
    dirname = ""
    for key, val in in_dict.iteritems():
        dirname += str(key) + "_" + str(val) + "_"
        
    return dirname[:-1]

In [4]:
def augment_config(mcoll, hyperparams, inputs_to_add, parent_dir):
    mconfhandler = ModelCollectionConfigFileHandler()
    mconfhandler.ToConfiguration(mcoll)
    
    # augment the original config
    mconfhandler.SetHyperparametersGlobally(hyperparams)
    
    added_params = 0
    # are there any input variables to add?
    for input_to_add in inputs_to_add:
        if input_to_add:
            added_params += 1
            mconfhandler.AddInputParameterGlobally(input_to_add)
        
    # convert to directory name
    dirname = dict_to_dirname(hyperparams) + "_extraparams_" + str(added_params)
     
    # make directory of that name
    confdir = parent_dir + dirname + "/"
    print confdir
    #if not os.path.exists(confdir):
    #    os.makedirs(confdir)
    
    # save the config file there
    #mconfhandler.SaveConfiguration(confdir + "settings.conf")

In [5]:
# performs a dynamic number of nested iterations
def iterate(start, end, step, fixed, callback):
    if len(start) == 0:
        return
    
    if len(start) > 1:
        # truncate the dict and keep going on the smaller one
        new_keys = start.keys()[1:]
        new_start = {key: start[key] for key in new_keys}
        new_end = {key: end[key] for key in new_keys}
        new_step = {key: step[key] for key in new_keys}
        
        new_fixed_key = start.keys()[0]
        
        for x in np.arange(start[new_fixed_key], end[new_fixed_key], step[new_fixed_key]):
            new_fixed = fixed
            new_fixed[new_fixed_key] = x
        
            iterate(new_start, new_end, new_step, new_fixed, callback)
    else:
        # only have one dimension left along which to iterate
        dimension = start.keys()[0]
        
        for x in np.arange(start[dimension], end[dimension], step[dimension]):
            allvars = fixed
            allvars[dimension] = x
            callback(allvars)

In [6]:
campaign_dir = "/data_CMS/cms/wind/testcampaign/"

In [7]:
confhandler = ConfigFileHandler()

In [22]:
confhandler.LoadConfiguration(campaign_dir + "campaign.conf")

attempting to load configuration file from /data_CMS/cms/wind/testcampaign/campaign.conf


In [9]:
available_sections = confhandler._get_sections()

In [10]:
# build the standard configuration from which to start
mcoll = ModelFactory.GenerateStandardModelCollections(SimpleModel, "/data_CMS/cms/wind/CJLST_NTuples/")

got the following list of hyperparams: {'number_layers': 2}
building network with inputs: ['PFMET', 'nCleanedJetsPt30', 'nCleanedJetsPt30BTagged_bTagSF', 'nExtraLep', 'D_VBF2j_ggH_ME']
got the following list of hyperparams: {'number_layers': 2}
building network with inputs: ['PFMET', 'nCleanedJetsPt30', 'nCleanedJetsPt30BTagged_bTagSF', 'nExtraLep', 'D_VBF1j_ggH_ME']
got the following list of hyperparams: {'number_layers': 2}
building network with inputs: ['PFMET', 'nCleanedJetsPt30', 'nCleanedJetsPt30BTagged_bTagSF', 'nExtraLep', 'D_WHh_ggH_ME']
got the following list of hyperparams: {'number_layers': 2}
building network with inputs: ['PFMET', 'nCleanedJetsPt30', 'nCleanedJetsPt30BTagged_bTagSF', 'nExtraLep', 'D_ZHh_ggH_ME']
got the following list of hyperparams: {'number_layers': 2}
building network with inputs: ['PFMET', 'nCleanedJetsPt30', 'nCleanedJetsPt30BTagged_bTagSF', 'nExtraLep', 'D_WHh_ZHh_ME']
got the following list of hyperparams: {'number_layers': 2}
building network with

In [23]:
# load the sweep parameters
start_dict = confhandler._process_dict(confhandler._get_field('hyperparameters', 'start'), lambda x: float(x))
end_dict = confhandler._process_dict(confhandler._get_field('hyperparameters', 'end'), lambda x: float(x))
step_dict = confhandler._process_dict(confhandler._get_field('hyperparameters', 'step'), lambda x: float(x))

In [24]:
# load the parameters
start_params = confhandler._process_list(confhandler._get_field('input_columns', 'start'), lambda x: x)
end_params = confhandler._process_list(confhandler._get_field('input_columns', 'end'), lambda x: x)

In [25]:
params_to_add = [param for param in end_params if param not in start_params]

In [26]:
params = start_params
iterate(start_dict, end_dict, step_dict, {}, lambda hyperparams: augment_config(mcoll, hyperparams, params, campaign_dir))
for param_to_add in params_to_add:
    params.append(param_to_add)
    iterate(start_dict, end_dict, step_dict, {}, lambda hyperparams: augment_config(mcoll, hyperparams, params, campaign_dir))

/data_CMS/cms/wind/testcampaign/number_layers_1.0_number_neurons_128.0_extraparams_0/
/data_CMS/cms/wind/testcampaign/number_layers_2.0_number_neurons_128.0_extraparams_0/
/data_CMS/cms/wind/testcampaign/number_layers_3.0_number_neurons_128.0_extraparams_0/
/data_CMS/cms/wind/testcampaign/number_layers_4.0_number_neurons_128.0_extraparams_0/
/data_CMS/cms/wind/testcampaign/number_layers_5.0_number_neurons_128.0_extraparams_0/
